In [1]:
import os
import numpy as np
os.chdir('utils')
from schroding_utils import *
from mlp_network import shared_model_func
from plotting import list_tensor_to_list, plot_loss, three_plots_schrod, heat_map_schrod
import pandas as pd
import time

In [2]:
# os.mkdir('../hyperparams_analysis')

In [3]:
# os.mkdir('../hyper_Schrod')

In [4]:
file_path = '../data/NLS.mat'

In [5]:
## Loading data
x, t, usol, usol_real, usol_img, usol_mag = load_data(file_path)

## Creating spatio-temporal grid
X, T = mesh_grid(x, t)

## Domain bounds
lb, ub = domain_bounds(-5.0, 5.0, 0.0, np.pi/2)

## Number of initial conditions training points
N0 = 50

## Number of boundary condition training points
N_b = 50

## Number of PDE structure (i.e. collocation) training points
N_f = 20000

## size of the neural network
## layers[0] == Input size
## layers[-1] == Output size
## layers[1:-1] == Number of units in between layers
layers = [2, 100, 100, 100, 100, 2]


## Preparing test data
X_test, y_u_test, y_v_test, y_h_test = test_data(X, T, usol_real, usol_img, usol_mag)

## Preparing training data
## choosing N0 random initial conditions
idx_x = np.random.choice(x.shape[0], N0, replace=False)
X_x_IC_train = x[idx_x,:] # spatial grid for train
X_u_IC_train = usol_real[idx_x,0:1] # real component for training
X_v_IC_train = usol_img[idx_x,0:1] # imaginary component for training

## chossing N_b boundary conditions
idx_t = np.random.choice(t.shape[0], N_b, replace=False)
X_tb_train = t[idx_t,:]

## Generating a latin-hypercube design
X_f_train = lb + (ub-lb)*lhs(2, N_f)


# Concating training arrays
X_IC_train = np.concatenate((X_x_IC_train, 0*X_x_IC_train), 1) 
X_lb_train = np.concatenate((0*X_tb_train + lb[0], X_tb_train), 1)
X_ub_train = np.concatenate((0*X_tb_train + ub[0], X_tb_train), 1)

In [6]:
## Here, we are investigating different optimizers, learning rate, depth and widht of the network.
## We are not changing the activation functions (just keeping the same as specified in the paper).

In [7]:
optim = ['Adam', 'RMSprop', 'SGD']
lr_rate = []
# Trying to perform random grid search by getting random learning rate
std_rate = [1e-3, 1e-2, 1e-1]
for i in range(3):
    a = np.round(np.random.uniform(0, 1, size=1)*std_rate[i], 4)
    lr_rate.append(a[0])
print("Learning rates used for hyparameters optimization are: {}\n".format(lr_rate))
depth = [2, 4, 6] # Number of layers
width = [100, 200, 300] # Number of neurons each layer
print("Total trials for the hyperparameters search are: {}\n".format(len(optim) * len(lr_rate) * len(depth) * len(width)))
print()  



Learning rates used for hyparameters optimization are: [0.0004, 0.0038, 0.0769]

Total trials for the hyperparameters search are: 81




In [8]:
def model_build_compile(x0, t0, x_lb, t_lb, x_ub, t_ub, x_f, t_f,shared_model, loss_function, optimizer, u0, v0):
    '''
    '''
    # Typicall plan, i.e. build model, compile model and fit model were not working in our case
    # We referred this link for training the model
    # Reference: https://www.tensorflow.org/guide/function
    # Reference: https://keras.io/api/optimizers/#learning-rate-decay--scheduling
    # Reference: https://colab.research.google.com/drive/1lo7Kf8zTb-DF_MjkO8Y07sYELnX3BNUR#scrollTo=GkimJNtepkKi
    # Reference: https://pgaleone.eu/tensorflow/tf.function/2019/03/21/dissecting-tf-function-part-1/
    # Reference: https://github.com/helonayala/pinn/blob/main/01_DeepVIV_sec21.ipynb
    # This apprach i.e. optmizers.minimize(cost/loss) is also mentioned in the original paper
    # Reference: https://www.tensorflow.org/guide/autodiff
    with tf.GradientTape() as tape_out:
        uv = shared_model([x0, t0], training=True)
        u0_pred = uv[:,0:1]
        v0_pred = uv[:,1:2]

        ## Loss function for initial conditon
        loss_1 = loss_function(u0_pred, u0) # Real part
        loss_2 = loss_function(v0_pred, v0) # Imaginary part
        
        x_lb = tf.convert_to_tensor(x_lb, dtype=tf.float64)
        x_ub = tf.convert_to_tensor(x_ub, dtype=tf.float64)
        x_f = tf.convert_to_tensor(x_f, dtype=tf.float64)
        t_f = tf.convert_to_tensor(t_f, dtype=tf.float64)
        with tf.GradientTape(persistent=True) as tape_1:
            tape_1.watch(x_lb)
            uv_lb = shared_model([x_lb, t_lb], training=True)
            u_lb_pred = uv_lb[:,0:1]
            v_lb_pred = uv_lb[:,1:2]
            u_x_lb_pred = tape_1.gradient(u_lb_pred, x_lb)
            v_x_lb_pred = tape_1.gradient(v_lb_pred, x_lb)


        with tf.GradientTape(persistent=True) as tape_2:
            tape_2.watch(x_ub)
            uv_ub = shared_model([x_ub, t_ub], training=True)
            u_ub_pred = uv_ub[:,0:1]
            v_ub_pred = uv_ub[:,1:2]
            u_x_ub_pred = tape_2.gradient(u_ub_pred, x_ub)
            v_x_ub_pred = tape_2.gradient(v_ub_pred, x_ub)

        ## Loss functions for the boundary conditions
        loss_3 = loss_function(u_lb_pred, u_ub_pred)
        loss_4 = loss_function(v_lb_pred, v_ub_pred)
        loss_5 = loss_function(u_x_lb_pred, u_x_ub_pred)
        loss_6 = loss_function(v_x_lb_pred, v_x_ub_pred)



        with tf.GradientTape(persistent=True) as tape_3:
            tape_3.watch(x_f)
            tape_3.watch(t_f)
            uv_f = shared_model([x_f, t_f], training=True)
            u_f = uv_f[:,0:1]
            v_f = uv_f[:,1:2]
            u_x_f = tape_3.gradient(u_f, x_f)
            v_x_f = tape_3.gradient(v_f, x_f)
            u_t_f = tape_3.gradient(u_f, t_f)
            v_t_f = tape_3.gradient(v_f, t_f)
        u_xx_f = tape_3.gradient(u_x_f, x_f)
        v_xx_f = tape_3.gradient(v_x_f, x_f)

        f_u_pred = u_t_f + 0.5 * v_xx_f + (u_f**2 + v_f**2)*v_f
        f_v_pred = v_t_f - 0.5 * u_xx_f - (u_f**2 + v_f**2)*u_f

        ## Loss function for the collocation points
        loss_7 = loss_function(f_u_pred, tf.convert_to_tensor(0, dtype=tf.float64))
        loss_8 = loss_function(f_v_pred, tf.convert_to_tensor(0, dtype=tf.float64))

        loss_combine = loss_1 + loss_2 + loss_3 + loss_4 + loss_5 + loss_6 + loss_7 + loss_8
    grads = tape_out.gradient(loss_combine, shared_model.trainable_weights)
    optimizer.apply_gradients(zip(grads, shared_model.trainable_weights))

    return loss_combine

In [9]:
def best_n_model(track_dict, best=20):
    new_dict = {}
    error_track = []
    best_model = []
    for k, v in track_dict.items():
        for k1,v1 in v.items():
            if k1 == 'error':
                error_track.append(v1)
                best_model.append(int(k))
    error_track_1, best_model_1 = zip(*sorted(zip(error_track, best_model)))
    for model in best_model_1[0:best]:
#         print("************************")
#         print("Best models are: \n")
#         print(track_dict['{}'.format(model)])
#         print("************************\n")
        new_dict['{}'.format(model)] = track_dict['{}'.format(model)]
    return new_dict 

In [10]:
# For different learning rates
def param_tuning(optim, lr_rate, depth, width, loss_function, train_dataset):
    track_dict = {}
    inp_num = 2
    out_num = 2
    klm = 0
    for opt in tqdm(optim):
        for lr in lr_rate:
            # For different number of layers in a network
            for dep in depth:
                # For different number of units in a layer
                for wid in width:
                    small_dict = {}
                    # Creating a list of size number of layers + 2
                    layers = [None]*(dep+2)
                    # Assigning size of input = 2
                    layers[0] = inp_num
                    # Assigning size of output = 1
                    layers[-1] = out_num
                    # Assigning number of neurons in hidden layers
                    layers[1:-1] = wid*np.ones(dep, dtype=int)
                    # Creating a model
                    shared_model = shared_model_func(layers, lb=None, ub=None, norm=False)
                    # Keeping the same loss function as specified in the paper
                    # Keeping the lear_rate)sched =. False
                    optimizer = choose_optimizer(lr=lr, ds=100, er=0.96, opt=opt, lear_rate_sched=False)
                    # Fixing maximum number of epochs 
                    max_Iter = 200
                    loss_model = []
                    start = time.time()
                    for epoch in range(max_Iter):
                        for (X_f_train1, t_f_train1) in train_dataset:   
                            # Passing the entire data in a single batch (That's how they did in the original paper)
                            loss_combine = model_build_compile(X_IC_train[:,0:1],
                                                                        X_IC_train[:,1:2], X_lb_train[:,0:1], 
                                                                        X_lb_train[:,1:2], X_ub_train[:,0:1], 
                                                                         X_ub_train[:,1:2], X_f_train[:,0:1], 
                                                                        X_f_train[:,1:2], shared_model, loss_function,
                                                                                    optimizer,X_u_IC_train, X_v_IC_train)

                            loss_model.append(loss_combine)
                    y_hat = predict(shared_model, X_test)
                    u_hat, v_hat = y_hat[:,0:1], y_hat[:,1:2]
                    h_hat = np.sqrt(u_hat**2 + v_hat**2)
                    
                    small_dict['optimizer'] = opt
                    small_dict['learning_rate'] = lr
                    small_dict['number of layer'] = dep
                    small_dict['number of neurons'] = wid
                    small_dict['loss model'] = list_tensor_to_list(loss_model)
                    small_dict['error'] = error_loss(y_h_test, h_hat)
                    track_dict['{}'.format(klm)] = small_dict
                    end = time.time()
                    if klm%9 == 0:
                        best_models = best_n_model(track_dict=track_dict, best=20)
                        df = pd.DataFrame.from_dict(track_dict,orient='index')
                        file = '../hyperparams_analysis/hyper_Schrod/dict_schrod'
                        df.to_csv(file)
                        
                    print("*******************************************************************************\n")
                    print("Trial {} has been completed".format(klm))
                    print("Time took to complete the trial is {}".format(end-start))
                    print("Total error in the solution is: {:e} ".format(error_loss(y_h_test, h_hat)))
                    print("Network trained is: {}".format(layers))
                    klm +=1
                    print("Details are: Optimizer: {0}, Learning Rate: {1}, Depth: {2}, Widht: {3}".format(opt, lr, dep, wid))
                    print("Deleting the current trained model")
                    del shared_model
                    del optimizer
                    del small_dict
                    del loss_model
#                     del loss_function
#                     del optimizer
                    print("\n*******************************************************************************\n")
    return track_dict






In [11]:
train_dataset = data_gen(X_f_train, batch_size=X_f_train.shape[0])
loss_function = tf.keras.losses.MeanSquaredError()
track_dict = param_tuning(optim, lr_rate, depth, width, loss_function, train_dataset)

  0%|          | 0/3 [00:00<?, ?it/s]

*******************************************************************************

Trial 0 has been completed
Time took to complete the trial is 27.14647388458252
Total error in the solution is: 7.082919e-01 
Network trained is: [2, 100, 100, 2]
Details are: Optimizer: Adam, Learning Rate: 0.0004, Depth: 2, Widht: 100
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial 1 has been completed
Time took to complete the trial is 49.865201473236084
Total error in the solution is: 6.887486e-01 
Network trained is: [2, 200, 200, 2]
Details are: Optimizer: Adam, Learning Rate: 0.0004, Depth: 2, Widht: 200
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial 2 has been completed
Time took to complete

*******************************************************************************

Trial 12 has been completed
Time took to complete the trial is 52.299259424209595
Total error in the solution is: 4.894907e-01 
Network trained is: [2, 100, 100, 100, 100, 2]
Details are: Optimizer: Adam, Learning Rate: 0.0038, Depth: 4, Widht: 100
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial 13 has been completed
Time took to complete the trial is 112.95425605773926
Total error in the solution is: 5.938431e-01 
Network trained is: [2, 200, 200, 200, 200, 2]
Details are: Optimizer: Adam, Learning Rate: 0.0038, Depth: 4, Widht: 200
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial 14 has been complet

 33%|███▎      | 1/3 [52:07<1:44:14, 3127.07s/it]

*******************************************************************************

Trial 26 has been completed
Time took to complete the trial is 281.9604752063751
Total error in the solution is: 4.904361e+00 
Network trained is: [2, 300, 300, 300, 300, 300, 300, 2]
Details are: Optimizer: Adam, Learning Rate: 0.0769, Depth: 6, Widht: 300
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial 27 has been completed
Time took to complete the trial is 27.19817304611206
Total error in the solution is: 6.681151e-01 
Network trained is: [2, 100, 100, 2]
Details are: Optimizer: RMSprop, Learning Rate: 0.0004, Depth: 2, Widht: 100
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial 28 has been comple

...

 67%|██████▋   | 2/3 [1:44:32<52:12, 3132.62s/it]

*******************************************************************************

Trial 53 has been completed
Time took to complete the trial is 283.0096571445465
Total error in the solution is: 2.769627e+00 
Network trained is: [2, 300, 300, 300, 300, 300, 300, 2]
Details are: Optimizer: RMSprop, Learning Rate: 0.0769, Depth: 6, Widht: 300
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial 54 has been completed
Time took to complete the trial is 27.170701026916504
Total error in the solution is: 8.130094e-01 
Network trained is: [2, 100, 100, 2]
Details are: Optimizer: SGD, Learning Rate: 0.0004, Depth: 2, Widht: 100
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial 55 has been comple

*******************************************************************************

Trial 72 has been completed
Time took to complete the trial is 26.500974655151367
Total error in the solution is: nan 
Network trained is: [2, 100, 100, 2]
Details are: Optimizer: SGD, Learning Rate: 0.0769, Depth: 2, Widht: 100
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial 73 has been completed
Time took to complete the trial is 48.954500675201416
Total error in the solution is: nan 
Network trained is: [2, 200, 200, 2]
Details are: Optimizer: SGD, Learning Rate: 0.0769, Depth: 2, Widht: 200
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial 74 has been completed
Time took to complete the trial is 72

100%|██████████| 3/3 [2:35:39<00:00, 3113.14s/it]

*******************************************************************************

Trial 80 has been completed
Time took to complete the trial is 272.73276257514954
Total error in the solution is: nan 
Network trained is: [2, 300, 300, 300, 300, 300, 300, 2]
Details are: Optimizer: SGD, Learning Rate: 0.0769, Depth: 6, Widht: 300
Deleting the current trained model

*******************************************************************************



In [12]:
best_models = best_n_model(track_dict=track_dict, best=20)
pd.DataFrame.from_dict(best_models,orient='index')

,optimizer,learning_rate,number of layer,number of neurons,loss model,error
6,Adam,0.0004,6,100,"[1.1165923082444351, 0.8455816553614568, 1.007...",0.426278
7,Adam,0.0004,6,200,"[2.5006211749550857, 3.2143473901419384, 2.130...",0.448586
3,Adam,0.0004,4,100,"[0.9783645654251814, 0.792291968610229, 0.8935...",0.449138
15,Adam,0.0038,6,100,"[1.1165923082444351, 125.65556447994095, 1.562...",0.453304
12,Adam,0.0038,4,100,"[0.9783645654251814, 176.59026860117592, 3.447...",0.489491
30,RMSprop,0.0004,4,100,"[0.9783645654251814, 4.207449835190346, 1.1490...",0.500646
9,Adam,0.0038,2,100,"[10.181119184275303, 1.915615249035909, 8.9229...",0.514169
4,Adam,0.0004,4,200,"[7.576681949673912, 0.9504553257659154, 1.9334...",0.522477
5,Adam,0.0004,4,300,"[3.1292152940295637, 4.164806443470297, 2.6175...",0.522576
33,RMSprop,0.0004,6,100,"[1.1165923082444351, 5.064906915416941, 1.2457...",0.533282


In [13]:
df_best = pd.DataFrame.from_dict(best_models,orient='index')
df_track = pd.DataFrame.from_dict(track_dict,orient='index')

In [14]:
file = '../hyperparams_analysis/hyper_Schrod/'
df_best.to_csv(file + 'best_dict')
df_track.to_csv(file + 'track_dict')

In [15]:
pd.read_csv(file+ 'best_dict')

,Unnamed: 0,optimizer,learning_rate,number of layer,number of neurons,loss model,error
0,6,Adam,0.0004,6,100,"[1.1165923082444351, 0.8455816553614568, 1.007...",0.426278
1,7,Adam,0.0004,6,200,"[2.5006211749550857, 3.2143473901419384, 2.130...",0.448586
2,3,Adam,0.0004,4,100,"[0.9783645654251814, 0.792291968610229, 0.8935...",0.449138
3,15,Adam,0.0038,6,100,"[1.1165923082444351, 125.65556447994095, 1.562...",0.453304
4,12,Adam,0.0038,4,100,"[0.9783645654251814, 176.59026860117592, 3.447...",0.489491
5,30,RMSprop,0.0004,4,100,"[0.9783645654251814, 4.207449835190346, 1.1490...",0.500646
6,9,Adam,0.0038,2,100,"[10.181119184275303, 1.915615249035909, 8.9229...",0.514169
7,4,Adam,0.0004,4,200,"[7.576681949673912, 0.9504553257659154, 1.9334...",0.522477
8,5,Adam,0.0004,4,300,"[3.1292152940295637, 4.164806443470297, 2.6175...",0.522576
9,33,RMSprop,0.0004,6,100,"[1.1165923082444351, 5.064906915416941, 1.2457...",0.533282


In [16]:
pd.read_csv(file+ 'track_dict')

,Unnamed: 0,optimizer,learning_rate,number of layer,number of neurons,loss model,error
0,0,Adam,0.0004,2,100,"[10.181119184275303, 7.168606627063127, 5.0295...",0.708292
1,1,Adam,0.0004,2,200,"[1.6177626703765782, 0.7242142060517835, 1.166...",0.688749
2,2,Adam,0.0004,2,300,"[4.990864720346963, 0.8818782704871069, 1.5152...",0.725500
3,3,Adam,0.0004,4,100,"[0.9783645654251814, 0.792291968610229, 0.8935...",0.449138
4,4,Adam,0.0004,4,200,"[7.576681949673912, 0.9504553257659154, 1.9334...",0.522477
...,...,...,...,...,...,...,...
76,76,SGD,0.0769,4,200,"[7.576681949673912, 75507.06133786985, 1.23138...",NaN
77,77,SGD,0.0769,4,300,"[3.1292152940295637, 615803.7256815056, inf, i...",NaN
78,78,SGD,0.0769,6,100,"[1.1165923082444351, 1604.0676949091503, 1.474...",NaN
79,79,SGD,0.0769,6,200,"[2.5006211749550857, 22.004259579523932, 16162...",NaN
